In [1]:
# import nessesary libaries

import os 
import pandas as pd
import numpy as np

import requests
import time
import json

try:
    from StringIO import StringIO
except ImportError:
    from io import StringIO
    
from datetime import datetime, timedelta

import http.client as httpclient
httpclient._MAXHEADERS = 100000

from scipy import stats

In [2]:
HOST = 'https://ch.itim.vn:443'
database = 'browser'
user = 'dungnn-stats'
password = 'g5aHu5M8'

<h3>Create functions</h3>

In [3]:
#get query results from DataBase
def get_clickhouse_data(query, host = HOST, connection_timeout = 60000):
    r = requests.post(host, params = {'query': query,
                                      'database': database,
                                      'user':user,
                                      'password':password},timeout = connection_timeout)
    if r.status_code == 200:
        return r.text
    else:
        raise ValueError(r.text)
        
# convert query results to pandas DataFrame       
def get_clickhouse_df(query, host = HOST, connection_timeout = 60000):
    data = get_clickhouse_data(query, host, connection_timeout) 
    df = pd.read_csv(StringIO(data), sep = '\t')
    return df

In [5]:
# join 2 tables
def left_join_tables(table1, table2, on_column):
    # Perform left join
    merged_table = pd.merge(table1, table2, on=on_column, how='left')
    return merged_table

# join multiple tables
def join_multiple_tables(tables, key_column):
    # Perform left join for each pair of tables
    result = tables[0]
    for table in tables[1:]:
        result = pd.merge(result, table, on=key_column, how='left')
    
    return result

<h4>Churn Data </h4>

In [13]:
query_00 = """
SELECT browser_id, 
	   if(tb2.browser_id_hash = 0, 1, 0) as churn_user
FROM
	(select browser_id, browser_id_hash
	from aggregated.browser_daily_profile
	where event_date between '2023-12-01' and '2023-12-31'
	and  os_name in ('windows', 'macos')
	group by browser_id, browser_id_hash
	) tb1
LEFT JOIN 
	(SELECT DISTINCT browser_id_hash
	FROM aggregated.browser_daily_profile bdp 
	WHERE event_date BETWEEN '2024-01-01' and '2024-01-31'
	) tb2
ON tb1.browser_id_hash = tb2.browser_id_hash
FORMAT TabSeparatedWithNames
"""

In [14]:
df_00 = get_clickhouse_df(query_00)

<h4>Demography data</h4>

In [6]:
# Age and Gender, location
query_01 = '''
select browser_id, 
		browser_id_hash, os_name,
		age_group, gender, 
        case when country = 'VN' or province <> '' then 'VN'
			 else country
		end as country, 
		region, 
		case when province in ('Hà Nội', 'Hồ Chí Minh', 'Cần Thơ', 'Hải Phòng', 'Đà Nẵng', 'Nha Trang', 'Khánh Hòa') then 'urban'
			 when province = '' then 'unknown'
			 else 'rural'
		end as province_type,
		province
from aggregated.browser_daily_profile
where event_date between '2023-12-01' and '2023-12-31'
and os_name in ('windows', 'macos')
limit 1 by browser_id
FORMAT TabSeparatedWithNames
'''

In [7]:
df_01 = get_clickhouse_df(query_01)

In [9]:
df_01.head()

,browser_id,browser_id_hash,os_name,age_group,gender,country,region,province_type,province
0,a7581279-7f3c-4d48-a347-41a91211df0f.s6v14i83i...,8908976142461698025,windows,25-34,male,VN,Southern Vietnam,rural,An Giang
1,a7582952-563e-421d-947e-4dd0f64df129.-1MycAVMV...,5168322062228518912,windows,25-34,female,VN,Central Vietnam,rural,Lâm Đồng
2,a7585e29-3a31-425f-842e-cbdc2c46d611.PJmD29x3h...,4367479955622191772,windows,18-24,female,VN,Northern Vietnam,urban,Hà Nội
3,a758c998-3e1e-4cba-8cf3-92096355fa09.sC8Z8GCXt...,13089603698493290699,windows,35-44,male,VN,Northern Vietnam,rural,Bắc Giang
4,a75ab1b5-97b7-4099-8ec7-d3693b013b2a._mgJsjDNX...,2037677313702276219,macos,18-24,female,VN,Northern Vietnam,urban,Hà Nội


In [15]:
print("df_01: ", len(df_01['browser_id'].unique()), len(df_01))
print("df_00: ", len(df_00['browser_id'].unique()), len(df_00))

df_01:  7902414 7902414
df_00:  7902414 7902414


In [16]:
table_list = [df_00, df_01]
result = join_multiple_tables(table_list, 'browser_id')

In [17]:
print("result: ", len(result['browser_id'].unique()), len(result))

result:  7902414 7902414


In [18]:
result.to_csv('desktop_data.csv', index = False)

In [21]:
# first day active
query_03 = '''
    select  browser_id, active_day,
		first_date, 
		toDate('2023-12-31') - first_date as life_time,
		lastest_active_day,
		toDate('2023-12-31') - lastest_active_day as not_active_day
    from 
    (select browser_id, browser_id_hash, max(event_date) as lastest_active_day, uniqExact(event_date) as active_day
    from aggregated.browser_daily_profile
    where event_date between '2023-12-01' and '2023-12-31'
    and os_name in ('windows', 'macos')
    group by browser_id, browser_id_hash
    ) t1
    left JOIN 
    (select browser_id_hash, first_date
    from aggregated.browser_first_date
    )
    t2
    on t1.browser_id_hash = t2.browser_id_hash
    FORMAT TabSeparatedWithNames
'''

In [22]:
df_03 = get_clickhouse_df(query_03)
df_03.head()

,browser_id,active_day,first_date,life_time,lastest_active_day,not_active_day
0,ABDB3866-071A-4B1A-9A2C-2AF9ED43234A.-lckIf_in...,25,2023-11-08,53,2023-12-30,1
1,F9B1821C-1CF5-4EF7-B3B9-CCF935302670.yBzIaK3yE...,30,2021-09-01,851,2023-12-31,0
2,00000000-0000-0000-0000-000000000001.gA8nTx73U...,28,2023-11-16,45,2023-12-31,0
3,3A5B7486-C30E-4EB7-B74F-500D79E83795.CP7tI6zBS...,3,2021-12-11,750,2023-12-24,7
4,06789DA1-6203-4873-B758-A136F793A0E6.jZdkJE5ib...,6,2023-07-19,165,2023-12-28,3


In [24]:
# There are some duplicates in browser_id --> we take only one row for each browser_id
# Convert 'life_time' column to numeric (if it contains mixed data types)
df_03['active_day'] = pd.to_numeric(df_03['active_day'], errors='coerce')

# Sort the DataFrame by 'life_time' in descending order
df_03_n = df_03.iloc[df_03['active_day'].astype(float).argsort()[::-1]]

# Keep only one row for each 'browser_id'
df_03_n = df_03_n.drop_duplicates(subset='browser_id', keep='first')

In [26]:
table_list = [result, df_03_n]
result = join_multiple_tables(table_list, 'browser_id')
print("result: ", len(result['browser_id'].unique()), len(result))

result:  7902414 7902414


In [28]:
result.to_csv('desktop_data.csv', index = False)

## Collect app event data

In [74]:
query_04 = '''
SELECT tb1.browser_id as browser_id,  newtab_count, download_count, pip_count, sidebar_count, incognito_count, signin_count
FROM
	(select browser_id, browser_id_hash
    from aggregated.browser_daily_profile
    where event_date = '{ddate}' 
    and os_name in ('windows', 'macos')
    group by browser_id, browser_id_hash
    ) tb1
LEFT JOIN
	(SELECT browser_id_hash, COUNT() newtab_count
	FROM frontend.log l 
	WHERE event_date = '{ddate}' 
		AND uri_webhpAction = 'Rendered' 
	GROUP BY browser_id_hash
	) tb2
ON tb1.browser_id_hash = tb2.browser_id_hash
LEFT JOIN 
	(select browser_id_hash, 
	   sumIf(count, name = 'Savior.Download' and min = 21) as download_count, 
	   sumIf(count, name = 'Savior.TryItNowButtonClicked' and min = 28) as pip_count,
	   sumIf(count, name in ('Sidebar.FeatureIcons.Clicked', 'Sidebar.CustomIcons.Clicked')) as sidebar_count,
	   sumIf(count, (name = 'Coccoc.NtpShow' and min = 1) or name = 'Tor.Tab.Count') as incognito_count,
	   sumIf(count, name = 'CoccocSync.Profile.NumberOfSignedInProfiles' and min = 0) as signin_count
	from browser_histograms.data 
	where event_date = '{ddate}' 
	and os_name in ('windows', 'macos')
	group by browser_id_hash
	) tb3
ON tb1.browser_id_hash = tb3.browser_id_hash
FORMAT TabSeparatedWithNames
'''

In [73]:
from datetime import datetime, timedelta

def date_range(start_date, end_date):
    date_list = []
    current_date = start_date
    while current_date <= end_date:
        date_list.append(current_date.strftime('%Y-%m-%d'))
        current_date += timedelta(days=1)
    return date_list

start_date = datetime(2023, 12, 1)  # Example start date
end_date = datetime(2023, 12, 31)    # Example end date
date_list = date_range(start_date, end_date)
print(date_list)

['2023-12-01', '2023-12-02', '2023-12-03', '2023-12-04', '2023-12-05', '2023-12-06', '2023-12-07', '2023-12-08', '2023-12-09', '2023-12-10', '2023-12-11', '2023-12-12', '2023-12-13', '2023-12-14', '2023-12-15', '2023-12-16', '2023-12-17', '2023-12-18', '2023-12-19', '2023-12-20', '2023-12-21', '2023-12-22', '2023-12-23', '2023-12-24', '2023-12-25', '2023-12-26', '2023-12-27', '2023-12-28', '2023-12-29', '2023-12-30', '2023-12-31']


In [75]:
numeric_columns = ['newtab_count', 'download_count', 'pip_count', 'sidebar_count', 'incognito_count', 'signin_count']

In [76]:
df_04 = pd.DataFrame()
for d in date_list:
    df_new = get_clickhouse_df(query_04.format(ddate = d))
    df_new[numeric_columns] = df_new[numeric_columns].apply(pd.to_numeric, errors='coerce')
    df_04 = pd.concat([df_04, df_new], axis=0, ignore_index=True)
    df_04 = df_04.groupby('browser_id')[numeric_columns].sum().reset_index()
    print("finish: ", d)

finish:  2023-12-01
finish:  2023-12-02
finish:  2023-12-03
finish:  2023-12-04
finish:  2023-12-05
finish:  2023-12-06
finish:  2023-12-07
finish:  2023-12-08
finish:  2023-12-09
finish:  2023-12-10
finish:  2023-12-11
finish:  2023-12-12
finish:  2023-12-13
finish:  2023-12-14
finish:  2023-12-15
finish:  2023-12-16
finish:  2023-12-17
finish:  2023-12-18
finish:  2023-12-19
finish:  2023-12-20
finish:  2023-12-21
finish:  2023-12-22
finish:  2023-12-23
finish:  2023-12-24
finish:  2023-12-25
finish:  2023-12-26
finish:  2023-12-27
finish:  2023-12-28
finish:  2023-12-29
finish:  2023-12-30
finish:  2023-12-31


In [77]:
df_04.head()

,browser_id,newtab_count,download_count,pip_count,sidebar_count,incognito_count,signin_count
0,00000000-0000-0000-0000-000000000000.-NwFcPRi5...,1,0,0,0,0,1
1,00000000-0000-0000-0000-000000000000.-_6s0KKv8...,0,0,0,0,0,4
2,00000000-0000-0000-0000-000000000000.05VCzf3rf...,0,0,0,0,0,0
3,00000000-0000-0000-0000-000000000000.0PyFy-34p...,0,0,0,0,0,0
4,00000000-0000-0000-0000-000000000000.0hIZkgULA...,0,0,0,0,0,0


## Collect web browsing data

In [41]:
query_05 = '''
	SELECT browser_id, sum(sum_duration) as total_active_time
	FROM aggregated.site_visits sv 
	where event_date between '2023-12-01' and '2023-12-31'
	and os_name in ('windows', 'macos')
	GROUP BY 1
    FORMAT TabSeparatedWithNames
    '''

In [42]:
df_05 = get_clickhouse_df(query_05)
df_05.head()

,browser_id,total_active_time
0,E8888A0B-21B4-411E-BCB8-ACC8B9201780.zRHE-9PS7...,10100
1,DAA2BA01-03CE-4A6C-98BB-991BC134A9CD.dUCaEFe_9...,30275
2,B4A39F5C-C47D-4416-8AFB-B421887C4930.9wZr3U60y...,26893
3,476D9145-8417-450D-9B81-DFC6A4CA01E7.-bG2F3OY5...,36822
4,b728dad3-5a86-4672-bc51-8b385b1f7d7e.oPi-CKxTq...,1314303


In [86]:
query_06_n = """
SELECT browser_id, 
		sumIf(clicks, category = 'Youtube-Music') as youtube_count,
		sumIf(clicks, category = 'Work-Study') as work_count,
		sumIf(clicks, category = 'Social') as social_count,
		sumIf(clicks, category = 'Info') as news_count,
		sumIf(clicks, category = 'Entertain') as entertainment_count,
		sumIf(clicks, category = 'Ecom') as ecommerce_count
FROM (
    SELECT browser_id,
    	domain,
    	sum(click_number) as clicks
    FROM aggregated.sites_clickstream
    WHERE event_date = yesterday() --'{ddate}'
	and os_name in ('windows', 'macos')
    GROUP BY 1,2
    ) d0
inner JOIN url('https://docs.google.com/spreadsheets/d/1G5nBXB02SljUqY3w7cqiYx5DwosP9y8JiZbv0aQ6b-c/gviz/tq?tqx=out:csv&sheet=Sheet1&range=A2:B', CSV, 'category String, domain String') cat 
    ON d0.domain = cat.domain
GROUP BY 1
FORMAT TabSeparatedWithNames
"""

In [87]:
numeric_columns = ['youtube_count', 'work_count', 'social_count', 'news_count', 'entertainment_count', 'ecommerce_count']

df_06_n = pd.DataFrame()
for d in date_list:
    df_new = get_clickhouse_df(query_06_n.format(ddate = d))
    df_new[numeric_columns] = df_new[numeric_columns].apply(pd.to_numeric, errors='coerce')
    df_06_n = pd.concat([df_06_n, df_new], axis=0, ignore_index=True)
    df_06_n = df_06_n.groupby('browser_id')[numeric_columns].sum().reset_index()
    print("finish: ", d)

finish:  2023-12-01
finish:  2023-12-02
finish:  2023-12-03
finish:  2023-12-04
finish:  2023-12-05
finish:  2023-12-06
finish:  2023-12-07
finish:  2023-12-08
finish:  2023-12-09
finish:  2023-12-10
finish:  2023-12-11
finish:  2023-12-12
finish:  2023-12-13
finish:  2023-12-14
finish:  2023-12-15
finish:  2023-12-16
finish:  2023-12-17
finish:  2023-12-18
finish:  2023-12-19
finish:  2023-12-20
finish:  2023-12-21
finish:  2023-12-22
finish:  2023-12-23
finish:  2023-12-24
finish:  2023-12-25
finish:  2023-12-26
finish:  2023-12-27
finish:  2023-12-28
finish:  2023-12-29
finish:  2023-12-30
finish:  2023-12-31


In [34]:
query_06 = """
    SELECT browser_id,
    	sum(click_number) as clicks
    FROM aggregated.sites_clickstream
    WHERE event_date between '2023-12-01' and '2023-12-31'
	AND os_name in ('windows', 'macos')
    GROUP BY 1
FORMAT TabSeparatedWithNames
"""

In [35]:
df_06 = get_clickhouse_df(query_06)
df_06.head()

,browser_id,clicks
0,190CA5E0-82BB-4C4D-B4A9-EC5E46436A24.uppRa7_W7...,6
1,DAA2BA01-03CE-4A6C-98BB-991BC134A9CD.dUCaEFe_9...,214
2,B4A39F5C-C47D-4416-8AFB-B421887C4930.9wZr3U60y...,304
3,476D9145-8417-450D-9B81-DFC6A4CA01E7.-bG2F3OY5...,752
4,b728dad3-5a86-4672-bc51-8b385b1f7d7e.oPi-CKxTq...,7452


<h4>Coccoc_search data</h4>

In [38]:
query_07 = """
select t1.browser_id as browser_id, search_volume, dating_search, videoclip_search, technical_search, 
		housekeeping_family_search, marketing_search,other_search,
		serp_click
from 
(select browser_id, browser_id_hash
from aggregated.browser_daily_profile
where event_date between '2023-12-01' and '2023-12-31'
and os_name in ('windows', 'macos')
group by browser_id, browser_id_hash
) t1
left join
(SELECT browser_id_hash,
		count() as search_volume,
		countIf(categories[1] = 15) as dating_search,
		countIf(categories[1] = 6) as videoclip_search,
		countIf(categories[1] = 19) as technical_search,
		countIf(categories[1] = 5) as housekeeping_family_search,
		countIf(categories[1]= 18) as marketing_search,
		countIf(categories[1]not in (5, 6, 15, 18, 19)) as other_search
FROM coccoc_search.classified c 
WHERE event_date between '2023-12-01' and '2023-12-31'
GROUP BY 1) 
t2
on t1.browser_id_hash = t2.browser_id_hash
left join
(SELECT browser_id_hash, count() as serp_click
FROM coccoc_search.serp_clicks sc 
WHERE event_date between '2023-12-01' and '2023-12-31'
GROUP BY 1
) t3
on t1.browser_id_hash = t3.browser_id_hash
FORMAT TabSeparatedWithNames
"""

In [39]:
df_07 = get_clickhouse_df(query_07)
df_07.head()

,browser_id,search_volume,dating_search,videoclip_search,technical_search,housekeeping_family_search,marketing_search,other_search,serp_click
0,ABDB3866-071A-4B1A-9A2C-2AF9ED43234A.-lckIf_in...,34,0,2,15,6,1,10,23
1,F9B1821C-1CF5-4EF7-B3B9-CCF935302670.yBzIaK3yE...,153,20,39,23,10,12,49,118
2,00000000-0000-0000-0000-000000000001.gA8nTx73U...,111,1,0,32,1,20,57,120
3,3A5B7486-C30E-4EB7-B74F-500D79E83795.CP7tI6zBS...,6,0,2,0,0,1,3,3
4,06789DA1-6203-4873-B758-A136F793A0E6.jZdkJE5ib...,36,13,4,8,0,2,9,52


<h4>Google search data</h4>

In [51]:
query_08 = """
SELECT tb1.browser_id as browser_id, search_volume, search_clicks,
	   other_search, housekeeping_family_search, videoclip_search, dating_search, marketing_search, technical_search
FROM
	(select browser_id, browser_id_hash
	from aggregated.browser_daily_profile
	where event_date between '2023-12-01' and '2023-12-31'
	and os_name in ('windows', 'macos')
	group by browser_id, browser_id_hash
	) tb1
LEFT JOIN 
	(SELECT browser_id_hash, COUNT(serp_id) as search_clicks  
	FROM google_search.serp_clicks sc 
	WHERE event_date between '2023-12-01' and '2023-12-31'
	GROUP BY 1
	) tb2
ON tb1.browser_id_hash = tb2.browser_id_hash
LEFT JOIN 
	(
	SELECT browser_id_hash,
		count() as search_volume,
		countIf(categories[1] = 15) as dating_search,
		countIf(categories[1] = 6) as videoclip_search,
		countIf(categories[1] = 19) as technical_search,
		countIf(categories[1] = 5) as housekeeping_family_search,
		countIf(categories[1]= 18) as marketing_search,
		countIf(categories[1]not in (5, 6, 15, 18, 19)) as other_search
	FROM google_search.classified
	WHERE event_date between '2023-12-01' and '2023-12-31'
	GROUP BY 1
	) tb5
ON tb1.browser_id_hash = tb5.browser_id_hash
FORMAT TabSeparatedWithNames
"""

In [52]:
df_08 = get_clickhouse_df(query_08)
df_08.head()

,browser_id,search_volume,search_clicks,other_search,housekeeping_family_search,videoclip_search,dating_search,marketing_search,technical_search
0,ABDB3866-071A-4B1A-9A2C-2AF9ED43234A.-lckIf_in...,4,0,2,0,0,0,0,2
1,F9B1821C-1CF5-4EF7-B3B9-CCF935302670.yBzIaK3yE...,259,88,120,14,102,3,2,18
2,00000000-0000-0000-0000-000000000001.gA8nTx73U...,3,0,1,0,0,0,1,1
3,3A5B7486-C30E-4EB7-B74F-500D79E83795.CP7tI6zBS...,1,0,0,0,0,0,1,0
4,06789DA1-6203-4873-B758-A136F793A0E6.jZdkJE5ib...,9,7,2,0,0,6,0,1


In [53]:
df_08.columns

Index(['browser_id', 'search_volume', 'search_clicks', 'other_search',
       'housekeeping_family_search', 'videoclip_search', 'dating_search',
       'marketing_search', 'technical_search'],
      dtype='object')

In [54]:
# Assuming df is your DataFrame
columns_to_rename = df_08.columns[df_08.columns != 'browser_id']
new_column_names = {column: f'{column}_gg' for column in columns_to_rename}

df_08 = df_08.rename(columns=new_column_names)

In [55]:
df_08.head()

,browser_id,search_volume_gg,search_clicks_gg,other_search_gg,housekeeping_family_search_gg,videoclip_search_gg,dating_search_gg,marketing_search_gg,technical_search_gg
0,ABDB3866-071A-4B1A-9A2C-2AF9ED43234A.-lckIf_in...,4,0,2,0,0,0,0,2
1,F9B1821C-1CF5-4EF7-B3B9-CCF935302670.yBzIaK3yE...,259,88,120,14,102,3,2,18
2,00000000-0000-0000-0000-000000000001.gA8nTx73U...,3,0,1,0,0,0,1,1
3,3A5B7486-C30E-4EB7-B74F-500D79E83795.CP7tI6zBS...,1,0,0,0,0,0,1,0
4,06789DA1-6203-4873-B758-A136F793A0E6.jZdkJE5ib...,9,7,2,0,0,6,0,1


In [91]:
table_list = [result, df_06_n]
result = join_multiple_tables(table_list, 'browser_id')

In [89]:
df_06_n.head()

,browser_id,youtube_count,work_count,social_count,news_count,entertainment_count,ecommerce_count
0,00000000-0000-0000-0000-000000000000.3IN_Vwh2V...,62,0,0,0,0,0
1,00000000-0000-0000-0000-000000000000.B9fRsX5M_...,744,341,11532,0,0,31
2,00000000-0000-0000-0000-000000000000.GW_QEOYIk...,248,0,0,0,0,0
3,00000000-0000-0000-0000-000000000000.Keo9HLTeg...,279,0,1736,0,0,0
4,00000000-0000-0000-0000-000000000000.ReToXhgXD...,0,0,31,31,0,0


In [92]:
result.head()

,browser_id,churn_user,browser_id_hash,os_name,age_group,gender,country,region,province_type,province,...,pip_count,sidebar_count,incognito_count,signin_count,youtube_count,work_count,social_count,news_count,entertainment_count,ecommerce_count
0,ABDB3866-071A-4B1A-9A2C-2AF9ED43234A.-lckIf_in...,0,2099492072614341214,windows,18-24,female,VN,Northern Vietnam,urban,Hà Nội,...,0,0,0,22,NaN,NaN,NaN,NaN,NaN,NaN
1,F9B1821C-1CF5-4EF7-B3B9-CCF935302670.yBzIaK3yE...,0,16568884414000652324,windows,15-17,female,VN,Northern Vietnam,rural,Thái Nguyên,...,0,97,4,27,NaN,NaN,NaN,NaN,NaN,NaN
2,00000000-0000-0000-0000-000000000001.gA8nTx73U...,0,2844554558337086803,windows,18-24,female,VN,Southern Vietnam,rural,Bình Dương,...,0,0,0,26,0.0,62.0,0.0,0.0,0.0,62.0
3,3A5B7486-C30E-4EB7-B74F-500D79E83795.CP7tI6zBS...,0,11210571368678352983,windows,18-24,male,VN,Northern Vietnam,rural,Vĩnh Phúc,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,06789DA1-6203-4873-B758-A136F793A0E6.jZdkJE5ib...,0,10549784423905206384,windows,25-34,male,VN,Northern Vietnam,urban,Hà Nội,...,0,5,0,6,NaN,NaN,NaN,NaN,NaN,NaN


In [93]:
result.to_csv('desktop_data.csv', index = False)

In [81]:
print("result ", len(result['browser_id'].unique()), len(result))

result  7902414 7902414


In [83]:
result['churn_user'].value_counts()


0    5969072
1    1933342
Name: churn_user, dtype: int64

<h4> Ads data</h4>

In [64]:
query_09 = """
SELECT browser_id,
		sum(impressions) as ads_impression,
		sum(clicks) as ads_click,
		sum(spending) as ads_revenue	
FROM aggregated.cc_qc_events
WHERE event_date between '2023-12-01' and '2023-12-31'
and os_name in ('windows', 'macos')
GROUP BY 1
FORMAT TabSeparatedWithNames
"""

In [65]:
df_09 = get_clickhouse_df(query_09)
df_09.head()

,browser_id,ads_impression,ads_click,ads_revenue
0,DAA2BA01-03CE-4A6C-98BB-991BC134A9CD.dUCaEFe_9...,100,0,524.500000
1,B4A39F5C-C47D-4416-8AFB-B421887C4930.9wZr3U60y...,207,0,1384.500000
2,476D9145-8417-450D-9B81-DFC6A4CA01E7.-bG2F3OY5...,826,0,3352.000000
3,b728dad3-5a86-4672-bc51-8b385b1f7d7e.oPi-CKxTq...,1701,0,9865.766926
4,957852a8-50e0-46a9-be0d-88998d21ed8a.apLE5MdW2...,1,0,0.000000


In [ ]:
# Clear memory
import gc

# List all DataFrames currently in memory
all_dfs = [var for var in globals() if isinstance(globals()[var], pd.DataFrame)]
print("DataFrames in memory:", all_dfs)


# Delete specific DataFrames
# For example, to delete df_01 and df_02
dfs_to_delete = ['df_09', 'df_04']

for df_name in dfs_to_delete:
    if df_name in globals():
        del globals()[df_name]

# Optionally, call garbage collection to free up memory
gc.collect()


In [96]:
result.shape

(7902414, 48)